In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)

# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/Data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder, '/content/drive/MyDrive/Real_Trader_EDA_R1/Data')  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['ES 09-24_Minute_240.csv',
 'ES 09-24_Minute_5.csv',
 'NQ 09-24_Minute_240.csv',
 'ES 09-24_Minute_60.csv',
 'NQ 09-24_Minute_5.csv',
 'NQ 09-24_Minute_60.csv',
 'ZB 09-24_Minute_240.csv',
 'ZB 09-24_Minute_5.csv',
 'ZB 09-24_Minute_60.csv',
 '.ipynb_checkpoints',
 'Updated_target.csv',
 'ES_5Min_Actual_Pred.csv',
 'New_5min_data (2).csv',
 'Actual_target.csv']

In [ ]:
# Define the function to update the DataFrame
def update_new_target(df):
    # Create a new column for New_Target
    df['New_Target'] = 0  # Initialize with None for better handling of nulls

    def update_entry(row):
        if row['Entry'] == 0:
            return 0  # Set New_Target to 0 if Entry is 0

        # Check BuyersInControlPrice condition
        if row['BuyersInControlPrice'] != 0:
            updated_entry = row['Entry'] + (row['Entry'] - row['Stop'])
            # Find values greater than updated_entry and sort them
            valid_values = sorted([row[col] for col in ['SellersLevel4', 'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
                                                       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4']
                                   if row[col] > updated_entry])

            if valid_values:
                return valid_values[0]  # Return the smallest value that is greater than updated_entry
            return updated_entry  # Return updated_entry if no valid values are found

        # Check SellersInControlPrice condition
        elif row['SellersInControlPrice'] != 0:
            updated_entry = row['Entry'] - (row['Stop'] - row['Entry'])
            # Find values less than updated_entry and sort them
            valid_values = sorted([row[col] for col in ['SellersLevel4', 'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
                                                       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3', 'BuyersLevel4']
                                   if row[col] < updated_entry], reverse=True)

            if valid_values:
                return valid_values[0]  # Return the largest value that is less than updated_entry
            return updated_entry  # Return updated_entry if no valid values are found

        return None

    # Apply the update_entry function to each row
    df['New_Target'] = df.apply(update_entry, axis=1)
    return df

In [ ]:
# Define a function to process all CSV files in a directory
def process_all_files(input_dir, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all CSV files in the input directory
    for file_path in glob.glob(os.path.join(input_dir, '*.csv')):
        # Read each file into a DataFrame
        df = pd.read_csv(file_path)

        # Drop unnecessary columns if present
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

        # Update the DataFrame using the logic defined
        updated_df = update_new_target(df)

        # Construct the output file path
        output_file_path = os.path.join(output_dir, os.path.basename(file_path))

        # Save the updated DataFrame to the output directory
        updated_df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

# Specify the input and output directories
input_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/Data/'
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/Updated_Data/'

# Process all files
process_all_files(input_directory, output_directory)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Real_Trader_EDA_R1/Sample/Updated_target (1).csv')
# df.rename(columns={'Time Stamp':'Time_Stamp'},inplace=True)
# df['Time_Stamp'] = pd.to_datetime(df['Time_Stamp'])
# df['Actual'] = df['Close']
# df['Difference'] = df.apply(lambda row: row['Actual'] - row['New_Target'] if row['New_Target'] != 0 else 0, axis=1)

In [ ]:
df.sample(20)

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target,New_Target
326223,09/16/2022 03:10,4231.50,4232.25,4230.50,4231.50,4236.00,4235.75,4235.50,4235.25,4232.75,4227.75,4228.00,4228.25,4229.00,4231.25,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00
241658,11/23/2021 11:10,5001.25,5005.25,4999.25,5005.25,5005.00,5004.50,5004.25,5003.75,5003.00,5000.00,5000.00,5000.50,5000.75,5001.00,0,0.00,11/23/2021 10:15,5025.25,5025.25,5026.25,5027.25,5023.25,5000.00
406322,06/29/2023 08:45,4661.00,4661.50,4658.00,4658.75,4663.50,4663.00,4663.00,4662.25,4662.00,4659.25,4659.25,4659.25,4659.25,4659.75,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00
205523,07/16/2021 07:00,4694.75,4696.00,4694.25,4695.50,4696.75,4696.50,4696.00,4696.00,4695.75,4692.25,4692.50,4692.75,4693.25,4694.00,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00
470147,02/15/2024 03:05,5152.75,5154.75,5152.50,5153.75,5154.00,5153.75,5153.75,5153.75,5153.50,5149.50,5150.25,5150.50,5151.50,5152.25,02/15/2024 02:15,5151.25,0,0.00,5151.25,5150.25,5149.25,5153.25,5154.00
476742,03/11/2024 03:55,5249.75,5253.50,5249.75,5253.00,5251.50,5251.25,5251.25,5251.00,5250.75,5247.00,5247.25,5247.25,5247.50,5247.75,03/11/2024 02:15,5246.50,0,0.00,5246.50,5245.50,5244.50,5248.50,5251.50
134581,10/29/2020 14:35,3639.25,3642.75,3636.75,3642.75,3642.25,3641.50,3640.50,3640.25,3639.50,3630.00,3631.00,3637.25,3637.75,3638.00,10/29/2020 14:15,3616.00,0,0.00,3616.00,3615.00,3614.00,3618.00,3642.25
41366,11/22/2019 07:50,3390.75,3390.75,3389.75,3390.50,3391.50,3391.25,3391.25,3391.00,3391.00,3389.25,3389.50,3389.50,3389.50,3390.00,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00
190511,05/21/2021 15:25,4489.75,4492.75,4489.00,4492.00,4492.50,4492.25,4492.00,4491.25,4490.00,4484.25,4484.25,4488.75,4488.75,4489.50,0,0.00,05/21/2021 15:15,4489.25,4489.25,4490.25,4491.25,4487.25,4484.25
499448,05/31/2024 06:20,5304.25,5304.25,5301.75,5301.75,5305.00,5304.75,5304.50,5304.50,5304.50,5303.50,5303.50,5304.00,5304.00,5304.00,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
# Define the function to process a single CSV file
def process_csv(file_path, output_dir):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Rename columns and process the DataFrame
    df.rename(columns={'Time Stamp': 'Time_Stamp'}, inplace=True)
    df['Time_Stamp'] = pd.to_datetime(df['Time_Stamp'])
    df['Actual'] = df['Close']
    df['Difference'] = df.apply(lambda row: row['Actual'] - row['New_Target'] if row['New_Target'] != 0 else 0, axis=1)

    # Get the base name of the file and create the output file path
    base_name = os.path.basename(file_path)
    output_file_path = os.path.join(output_dir, base_name)

    # Save the updated DataFrame to the output file path
    df.to_csv(output_file_path, index=False)

# Specify the directory containing the CSV files
input_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/Sample/'

# Specify the directory to save updated CSV files
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/Updated_Data/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Iterate over all CSV files in the input directory
for file_path in glob.glob(os.path.join(input_directory, '*.csv')):
    # Process each CSV file and save the updated file in the output directory
    process_csv(file_path, output_directory)

print("Processing complete. Updated files are saved in:", output_directory)

Processing complete. Updated files are saved in: /content/drive/MyDrive/Real_Trader_EDA_R1/Updated_Data/


In [ ]:
s = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Updated_Data/Copy of Updated_target.csv")

In [ ]:
s[["Close","New_Target","Actual","Difference"]].sample(20)

,Close,New_Target,Actual,Difference
139778,3917.75,0.00,3917.75,0.00
243345,4960.25,4940.25,4960.25,20.00
174380,4234.75,4222.00,4234.75,12.75
153380,4069.50,4064.75,4069.50,4.75
458366,4875.50,4867.00,4875.50,8.50
333196,3951.75,3933.50,3951.75,18.25
146297,3987.50,0.00,3987.50,0.00
33558,3290.25,3291.25,3290.25,-1.00
495649,5383.00,0.00,5383.00,0.00
372469,4291.00,0.00,4291.00,0.00


In [ ]:
# Define a function to process all CSV files in a directory
def process_all_files(input_dir, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over all CSV files in the input directory
    for file_path in glob.glob(os.path.join(input_dir, '*.csv')):
        # Read each file into a DataFrame
        df = pd.read_csv(file_path)

        # Drop unnecessary columns if present
        if 'Unnamed: 0' in df.columns:
            df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

        # Update the DataFrame using the logic defined
        updated_df = update_new_target(df)

        # Construct the output file path
        output_file_path = os.path.join(output_dir, os.path.basename(file_path))

        # Save the updated DataFrame to the output directory
        updated_df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

# Specify the input and output directories
input_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/Data/'
output_directory = '/content/drive/MyDrive/Real_Trader_EDA_R1/Updated_Data/'

# Process all files
process_all_files(input_directory, output_directory)

In [ ]:
import pandas as pd

def update_targets(df):
    # Initialize columns and variables
    df['New_Target'] = 0
    current_high = None
    current_low = None
    consecutive_below_count = 0

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        entry_value = row['Entry']
        close_value = row['Close']
        stop_value = row['Stop']
        target_value = row['Target']

        # Check if we are tracking the highest or lowest value based on the target
        if target_value == row['BuyersInControlPrice']:
            # Track the highest value if BuyersInControlPrice is present
            if current_high is None:
                current_high = close_value
            else:
                current_high = max(current_high, close_value)

        if target_value == row['SellersInControlPrice']:
            # Track the lowest value if SellersInControlPrice is present
            if current_low is None:
                current_low = close_value
            else:
                current_low = min(current_low, close_value)

        # Check if Close value matches the Stop value
        if close_value == stop_value:
            consecutive_below_count += 1
        else:
            consecutive_below_count = 0

        # Update New_Target based on the conditions
        if consecutive_below_count >= 3:
            if current_high is not None:
                df.at[index, 'New_Target'] = current_high
            elif current_low is not None:
                df.at[index, 'New_Target'] = current_low
            else:
                df.at[index, 'New_Target'] = entry_value
            # Reset tracking variables after updating
            current_high = None
            current_low = None
            consecutive_below_count = 0
        else:
            # Set the target to the most recent valid current_high or current_low if not yet updated
            if current_high is not None:
                df.at[index, 'New_Target'] = current_high
            elif current_low is not None:
                df.at[index, 'New_Target'] = current_low
            else:
                df.at[index, 'New_Target'] = entry_value

    return df

# Sample DataFrame for testing
data = {
    'Time Stamp': ['2024-07-01', '2024-07-02', '2024-07-03', '2024-07-04', '2024-07-05'],
    'Open': [100, 102, 104, 103, 101],
    'High': [105, 106, 107, 108, 106],
    'Low': [99, 101, 103, 100, 98],
    'Close': [103, 105, 102, 101, 100],
    'SellersLevel4': [110, 110, 110, 110, 110],
    'SellersLevel3': [105, 105, 105, 105, 105],
    'SellersLevel2': [100, 100, 100, 100, 100],
    'SellersLevel1': [95, 95, 95, 95, 95],
    'SellersLevel0': [90, 90, 90, 90, 90],
    'BuyersLevel0': [85, 85, 85, 85, 85],
    'BuyersLevel1': [90, 90, 90, 90, 90],
    'BuyersLevel2': [95, 95, 95, 95, 95],
    'BuyersLevel3': [100, 100, 100, 100, 100],
    'BuyersLevel4': [105, 105, 105, 105, 105],
    'BuyersInControlTime': [1, 1, 1, 1, 1],
    'BuyersInControlPrice': [105, 105, 105, 105, 105],
    'SellersInControlTime': [1, 1, 1, 1, 1],
    'SellersInControlPrice': [95, 95, 95, 95, 95],
    'Entry': [101, 102, 103, 104, 105],
    'Defence': [100, 101, 102, 103, 104],
    'Stop': [100, 100, 100, 100, 100],
    'Target': [105, 106, 107, 108, 109]
}

df = pd.DataFrame(data)
df = update_targets(df)


In [ ]:
df

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target,New_Target
0,2024-07-01,100,105,99,103,110,105,100,95,90,85,90,95,100,105,1,105,1,95,101,100,100,105,103
1,2024-07-02,102,106,101,105,110,105,100,95,90,85,90,95,100,105,1,105,1,95,102,101,100,106,103
2,2024-07-03,104,107,103,102,110,105,100,95,90,85,90,95,100,105,1,105,1,95,103,102,100,107,103
3,2024-07-04,103,108,100,101,110,105,100,95,90,85,90,95,100,105,1,105,1,95,104,103,100,108,103
4,2024-07-05,101,106,98,100,110,105,100,95,90,85,90,95,100,105,1,105,1,95,105,104,100,109,103


In [ ]:
s = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Data/New_5min_data (2).csv")

In [ ]:
s.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
s.s

In [ ]:
import pandas as pd
import numpy as np

def update_targets(df):
    # Initialize columns and variables
    df['New_Target'] = np.nan  # Use NaN as the placeholder for no value
    current_high = None
    current_low = None
    consecutive_below_count = 0

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        entry_value = row['Entry']
        close_value = row['Close']
        stop_value = row['Stop']
        target_value = row['Target']

        # Check if we are tracking the highest or lowest value based on the target
        if target_value == row['BuyersInControlPrice']:
            # Track the highest value if BuyersInControlPrice is present
            if current_high is None:
                current_high = close_value
            else:
                current_high = max(current_high, close_value)

        if target_value == row['SellersInControlPrice']:
            # Track the lowest value if SellersInControlPrice is present
            if current_low is None:
                current_low = close_value
            else:
                current_low = min(current_low, close_value)

        # Check if Close value matches the Stop value
        if close_value == stop_value:
            consecutive_below_count += 1
        else:
            consecutive_below_count = 0

        # Update New_Target based on the conditions
        if consecutive_below_count >= 3:
            if current_high is not None:
                df.at[index, 'New_Target'] = current_high
            elif current_low is not None:
                df.at[index, 'New_Target'] = current_low
            # Reset tracking variables after updating
            current_high = None
            current_low = None
            consecutive_below_count = 0
        else:
            # Set the target to the most recent valid current_high or current_low if not yet updated
            if current_high is not None:
                df.at[index, 'New_Target'] = current_high
            elif current_low is not None:
                df.at[index, 'New_Target'] = current_low
            # If neither current_high nor current_low is available, keep New_Target as NaN

    return df

In [ ]:
import pandas as pd

def update_high_actual(df):
    """
    Update the 'high_Actual' column in the DataFrame based on the maximum 'Close' value
    before the 'Stop' value and compare it with the 'Target'.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Close', 'Stop', 'Target', and 'high_Actual' columns.

    Returns:
    pd.DataFrame: Updated DataFrame with 'high_Actual' column set appropriately.
    """
    # Ensure 'high_Actual' column exists and initialize with zeros
    if 'high_Actual' not in df.columns:
        df['high_Actual'] = 0

    # Initialize variables
    max_close_before_stop = None
    stop_found = False

    # Iterate through DataFrame
    for i in range(len(df)):
        if df.loc[i, 'Stop'] != 0:
            stop_found = True

        if not stop_found or df.loc[i, 'Stop'] == 0:
            if max_close_before_stop is None or df.loc[i, 'Close'] > max_close_before_stop:
                max_close_before_stop = df.loc[i, 'Close']

    # If a valid max_close_before_stop was found and Stop was encountered
    if max_close_before_stop is not None:
        # Find Target index
        target_index = df[df['Target'] != 0].index
        if not target_index.empty:
            target_index = target_index[0]

            # Update high_Actual if max_close_before_stop is greater than or equal to Target
            if max_close_before_stop >= df.loc[target_index, 'Target']:
                df.loc[target_index, 'high_Actual'] = max_close_before_stop

    return df

# Sample DataFrame
data = {
    'Close': [103, 104, 105, 106, 107, 109, 102, 100],
    'Stop': [100, 0, 0, 0, 102, 0, 0, 0],
    'Target': [105, 0, 0, 106, 0, 0, 0, 0],
}

df = pd.DataFrame(data)

# Apply the function to the DataFrame
updated_df = update_high_actual(df)
print(updated_df)

   Close  Stop  Target  high_Actual
0    103   100     105          109
1    104     0       0            0
2    105     0       0            0
3    106     0     106            0
4    107   102       0            0
5    109     0       0            0
6    102     0       0            0
7    100     0       0            0


In [52]:
import pandas as pd

def update_high_actual(df):
    """
    Update the 'high_Actual' column in the DataFrame based on the maximum 'Close' value
    before each 'Stop' value. If no exact 'Close' value matches the 'Stop', use the most
    recent 'Close' value below the 'Stop'. This is applied only for rows where both 'Target'
    and 'Stop' values are present in the same row.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Close', 'Stop', 'Target', and 'high_Actual' columns.

    Returns:
    pd.DataFrame: Updated DataFrame with 'high_Actual' column set appropriately.
    """
    # Initialize 'high_Actual' column
    df['high_Actual'] = 0

    # Initialize variables
    max_close_before_stop = -float('inf')
    recent_close_below_stop = -float('inf')

    # Iterate through DataFrame
    for i in range(len(df)):
        current_close = df.loc[i, 'Close']
        current_stop = df.loc[i, 'Stop']
        current_target = df.loc[i, 'Target']

        # If Stop value is encountered
        if current_stop != 0:
            # Update recent_close_below_stop if available
            if recent_close_below_stop > -float('inf'):
                max_close_before_stop = max(max_close_before_stop, recent_close_below_stop)
            recent_close_below_stop = -float('inf')  # Reset for next potential Stop
        else:
            # Update recent_close_below_stop if current Close is below the most recent Stop
            if recent_close_below_stop < -float('inf') or current_close < recent_close_below_stop:
                recent_close_below_stop = current_close

        # Update max_close_before_stop if current Close is valid
        if current_close > max_close_before_stop:
            max_close_before_stop = current_close

        # Assign to high_Actual where both Target and Stop are present
        if current_target != 0 and current_stop != 0:
            if max_close_before_stop >= current_target:
                df.loc[i, 'high_Actual'] = max_close_before_stop
            elif recent_close_below_stop >= current_target:
                df.loc[i, 'high_Actual'] = recent_close_below_stop

    return df

# Sample DataFrame with your data
data = {
    'Close': [103, 104, 105, 106, 107, 103, 102, 100, 108, 109,
              115, 110, 120, 130, 140, 125, 135],
    'Stop': [0, 0, 100, 0, 0, 102, 0, 101, 0, 0,
             108, 0, 130, 125, 0, 130, 110],
    'Target': [0, 0, 105, 0, 0, 110, 0, 120, 0, 0,
               125, 0, 135, 140, 0, 145, 150],
}

df = pd.DataFrame(data)

# Apply the function to the DataFrame
updated_df = update_high_actual(df)
print(updated_df)


    Close  Stop  Target  high_Actual
0     103     0       0            0
1     104     0       0            0
2     105   100     105          105
3     106     0       0            0
4     107     0       0            0
5     103   102     110            0
6     102     0       0            0
7     100   101     120            0
8     108     0       0            0
9     109     0       0            0
10    115   108     125            0
11    110     0       0            0
12    120   130     135            0
13    130   125     140            0
14    140     0       0            0
15    125   130     145            0
16    135   110     150            0


In [56]:
import pandas as pd

def update_high_actual(df):
    """
    Update the 'high_Actual' column based on the maximum 'Close' value before each 'Stop' value.
    If no exact 'Close' value matches the 'Stop', use the most recent 'Close' value below the 'Stop'.
    This is applied only for rows where both 'Target' and 'Stop' values are present in the same row.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Close', 'Stop', 'Target', and 'high_Actual' columns.

    Returns:
    pd.DataFrame: Updated DataFrame with 'high_Actual' column set appropriately.
    """
    # Ensure 'high_Actual' column exists and initialize with zeros
    if 'high_Actual' not in df.columns:
        df['high_Actual'] = 0

    # Initialize variables
    max_close_before_stop = None
    recent_close_below_stop = None

    # Iterate through DataFrame
    for i in range(len(df)):
        # If Stop value is present, find the maximum 'Close' value before this Stop
        if df.loc[i, 'Stop'] != 0:
            # Find the maximum 'Close' value up to and including the current row
            max_close_before_stop = df.loc[:i, 'Close'].max()
            # If Stop value is greater than or equal to current Close, update recent_close_below_stop
            if df.loc[i, 'Close'] <= df.loc[i, 'Stop']:
                recent_close_below_stop = df.loc[i, 'Close']
            # Update recent_close_below_stop to the highest value less than Stop
            if df.loc[i, 'Close'] < df.loc[i, 'Stop']:
                recent_close_below_stop = df.loc[i, 'Close']
        else:
            # Update recent_close_below_stop if Close is below the last Stop value
            if recent_close_below_stop is None or df.loc[i, 'Close'] < recent_close_below_stop:
                recent_close_below_stop = df.loc[i, 'Close']

        # Update high_Actual where both Target and Stop are present
        if df.loc[i, 'Target'] != 0 and df.loc[i, 'Stop'] != 0:
            # Use max_close_before_stop if available and valid
            if max_close_before_stop is not None and max_close_before_stop >= df.loc[i, 'Target']:
                df.loc[i, 'high_Actual'] = max_close_before_stop
            # Otherwise, use recent_close_below_stop if it's valid
            elif recent_close_below_stop is not None and recent_close_below_stop >= df.loc[i, 'Target']:
                df.loc[i, 'high_Actual'] = recent_close_below_stop

    return df

# Sample DataFrame with your data
data = {
    'Close': [103, 104, 105, 106, 107, 103, 102, 100, 108, 109,
              115, 110, 120, 130, 140, 125, 135],
    'Stop': [0, 0, 100, 0, 0, 102, 0, 101, 0, 0,
             108, 0, 130, 125, 0, 130, 110],
    'Target': [0, 0, 105, 0, 0, 110, 0, 120, 0, 0,
               125, 0, 135, 140, 0, 145, 150],
}

df = pd.DataFrame(data)

# Apply the function to the DataFrame
updated_df = update_high_actual(df)
print(updated_df)


    Close  Stop  Target  high_Actual
0     103     0       0            0
1     104     0       0            0
2     105   100     105          105
3     106     0       0            0
4     107     0       0            0
5     103   102     110            0
6     102     0       0            0
7     100   101     120            0
8     108     0       0            0
9     109     0       0            0
10    115   108     125            0
11    110     0       0            0
12    120   130     135            0
13    130   125     140            0
14    140     0       0            0
15    125   130     145            0
16    135   110     150            0


In [ ]:
# Sample DataFrame with updated data
data = {
    'Close': [103, 104, 105, 106, 107, 103, 102, 100, 108, 109,
              115, 110, 120, 130, 140, 125, 135, 125, 140, 145,
              150, 155, 160, 165, 170, 175, 180, 185, 190, 195],
    'Stop': [0, 0, 100, 0, 0, 102, 0, 101, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Target': [0, 0, 105, 0, 0, 110, 0, 120, 0, 0,
               125, 0, 135, 140, 0, 145, 150, 0, 0, 0,
               160, 0, 0, 0, 165, 0, 0, 0, 0, 195],
}

df = pd.DataFrame(data)

In [ ]:
df

,Close,Stop,Target
0,103,0,0
1,104,0,0
2,105,100,105
3,106,0,0
4,107,0,0
5,103,102,110
6,102,0,0
7,100,101,120
8,108,0,0
9,109,0,0


In [3]:
import pandas as pd

def update_high_actual_with_target_and_stop(df):
    """
    Update the 'high_Actual' column in the DataFrame based on the maximum 'Close' value
    before each 'Stop' value. If no exact 'Close' value matches the 'Stop', use the most
    recent 'Close' value below the 'Stop'. This is applied only for rows where both 'Target'
    and 'Stop' values are present in the same row.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Close', 'Stop', 'Target', and 'high_Actual' columns.

    Returns:
    pd.DataFrame: Updated DataFrame with 'high_Actual' column set appropriately.
    """
    # Ensure 'high_Actual' column exists and initialize with zeros
    if 'high_Actual' not in df.columns:
        df['high_Actual'] = 0

    # Initialize variables
    max_close_before_stop = 0
    recent_close_below_stop = 0

    # Iterate through DataFrame
    for i in range(len(df)):
        if df.loc[i, 'Stop'] != 0 and df.loc[i, 'Target'] != 0:
            # Find the max Close value from the start to the point just before the Stop value
            max_close_before_stop = 0
            for j in range(i):
                if df.loc[j, 'Close'] <= df.loc[i, 'Stop']:
                    recent_close_below_stop = df.loc[j, 'Close']
                if df.loc[j, 'Close'] > max_close_before_stop:
                    max_close_before_stop = df.loc[j, 'Close']

            # Update high_Actual where both Target and Stop are present
            df.loc[i, 'high_Actual'] = max(max_close_before_stop, recent_close_below_stop)

    return df

# Sample DataFrame with your data
data = {
    'Close': [103, 104, 105, 106, 107, 103, 102, 100, 108, 109,
              115, 110, 120, 130, 140, 125, 135],
    'Stop': [0, 0, 100, 0, 0, 102, 0, 101, 0, 0,
             108, 0, 130, 125, 0, 130, 110],
    'Target': [0, 0, 105, 0, 0, 110, 0, 120, 0, 0,
               125, 0, 135, 140, 0, 145, 150],
}

df = pd.DataFrame(data)

# Apply the function to the DataFrame
updated_df = update_high_actual_with_target_and_stop(df)
print(updated_df)


    Close  Stop  Target  high_Actual
0     103     0       0            0
1     104     0       0            0
2     105   100     105          104
3     106     0       0            0
4     107     0       0            0
5     103   102     110          107
6     102     0       0            0
7     100   101     120          107
8     108     0       0            0
9     109     0       0            0
10    115   108     125          109
11    110     0       0            0
12    120   130     135          115
13    130   125     140          120
14    140     0       0            0
15    125   130     145          140
16    135   110     150          140
